# Welcome to facemap

 Login to the facemap node app and start the session

In [9]:
import sys

In [10]:
# Here are all the installs and imports you will need for facemap to run.
    
!{sys.executable} -m pip install requests
# !{sys.executable} -m pip install socketIO-client
!{sys.executable} -m pip install opencv-python pip install pillow
!{sys.executable} -m pip install pillow
!{sys.executable} -m pip install "python-socketio[client]"


In [11]:
import requests
import socketio
import cv2
import os
import numpy as np
from PIL import Image
import pickle
import random

In [12]:
# print(dir(cv2.face))
# 'C:\\Users\\17211\\AppData\\Roaming\\Python\\Python37\\site-packages\\cv2\\cv2.cp37-win_amd64.pyd'
print(cv2.__file__)

C:\Users\17211\AppData\Roaming\Python\Python38\site-packages\cv2\cv2.cp38-win_amd64.pyd


In [13]:
TOKEN =""

_isLocal = False;

_baseUrl="https://face-map-node-server.herokuapp.com/api/";
_host = "https://face-map-node-server.herokuapp.com/"

if(_isLocal):
    _baseUrl="http://localhost:1291/api/";
    _host = "http://localhost:1291"
    
    
# https://face-map-node-server.herokuapp.com/



login_url =_baseUrl+"user/dashboard/"
register_url =_baseUrl+"user/register/"

post_url = _baseUrl + "posts/"

In [14]:
# start session

pload = {'username':'facemap_admin','password':'facemap_admin1291'}

if(_isLocal):
    pload = {'username':'puneeth','password':'password'}

# register = requests.post(register_url,data=pload,verify=True)

# print(register.json())
# print("___________ Registered In ___________")



res = requests.post(login_url,data=pload, verify=True);
if(res.status_code==200):
    TOKEN= res.headers['auth-token'];
    print("___________ Logged In ___________"+TOKEN)
locations = ["Library","IT - BLOCK","CSE - BLOCK"]
for obj in res.json()['data']["location"]:
    locations.append(obj["name"]);


print(res.json())



___________ Logged In ___________eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJfaWQiOiI1ZWE3MmZkNDM0MmVlYjNlNjgwMGI5MTUiLCJpYXQiOjE2MTQwMDE5NTN9.Tw00zPk4GgLQ3Wbeq3jjDUETZfVQKRVWd-4tm2-Ymps
{'status': 'success', 'message': 'Logged In', 'data': {'location': [{'createdAt': '2021-01-07T04:30:25.592Z', '_id': '5ff68e61eb22cc001747b917', 'name': 'Dosa point', 'lat': '17.724984', 'long': '78.255608', '__v': 0}, {'createdAt': '2021-01-07T04:30:28.393Z', '_id': '5ff68e64eb22cc001747b918', 'name': 'Library', 'lat': '17.726076', 'long': '78.256657', '__v': 0}, {'createdAt': '2021-01-07T04:30:28.595Z', '_id': '5ff68e64eb22cc001747b919', 'name': 'Swami Vivekananda Boys Hostel', 'lat': '17.724278', 'long': '78.2539', '__v': 0}, {'createdAt': '2021-01-07T04:30:28.795Z', '_id': '5ff68e64eb22cc001747b91a', 'name': 'Mahatma Gandhi Statue', 'lat': '17.725062', 'long': '78.254761', '__v': 0}, {'createdAt': '2021-01-07T04:30:28.993Z', '_id': '5ff68e64eb22cc001747b91b', 'name': 'C V Raman Boys Hostel', 'lat': '17

In [15]:
class Recognizer:
    
    def __init__(self,socket):
        self.socket =socket
        self.stop = False
        print("initializing Recognizer")
        pass
    
    def disconnect(self):
        self.socket.disconnect()

    

    
    def train(self):
        try:
            print("training ")
            # BASE_DIR = os.path.dirname(os.path.abspath(__file__)) this says where is this notebook exactly is..

            recognizer = cv2.face.LBPHFaceRecognizer_create()

            faces_got,ids = self.getImageAndLabels("./dataset/")
            recognizer.train(faces_got, np.array(ids))

            # Save the model into recognizers/trainer.yml
            recognizer.write('recognizers/trainner.yml')

            print("Saved the model into recognizers/trainer.yml")
        except Exception as e:
            print(e)
        
    def getImageAndLabels(self,path):
            face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_default.xml')
            
            print("getting all face_vectors and face_labels")
            imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
        #     print(len(imagePaths))
            faceSamples=[]
            ids = []
            for root, dirs, files_names in os.walk(path):
                total_files = 0
                detected_faces = 0
                if(root==path):
                    continue
                for file in files_names:
                    if file.endswith("png") or file.endswith("jpg") or file.endswith("JPG"):
                        total_files = total_files+1
                        image_path = os.path.join(root, file)
                        PIL_img = Image.open(image_path).convert('L') # convert it to grayscale
                        img_numpy = np.array(PIL_img,'uint8')

                        # print(os.path.split(file)) ('', '1.194.jpg')
                        # os.path.split(file)[-1].split(".") --->>> ['1', '194', 'jpg']

                        label_id = int(os.path.split(file)[-1].split(".")[0])
                        faces = face_cascade.detectMultiScale(img_numpy) 

                        for (x,y,w,h) in faces:
                            detected_faces = detected_faces+1
                            faceSamples.append(img_numpy[y:y+h,x:x+w])
                            ids.append(label_id)
                print("root:",root,end='\n')
                print("DIR:",len(dirs),end='\n')
                print("Files:",len(files_names))
                print("total images:"+str(total_files))
                print("detected_faces:" +str(detected_faces))
            return faceSamples,ids
        

        
    def recognize(self):
        
        try:
            recognizer = cv2.face.LBPHFaceRecognizer_create()
#             recognizer.read('recognizers/trainner.yml')
            recognizer.read('recognizers/trainner_3_but_better.yml')

            faceCascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt.xml')
            font = cv2.FONT_HERSHEY_SIMPLEX

            id = 0
            names = ['', '', 'Priyanka Chopra',"Puneeth","Tiger Shroff","Vicky Kaushal","Shah_Rukh_Khan"]
#             url = input()
            cam = cv2.VideoCapture(0)
            cam.set(3, 640) # set video widht
            cam.set(4, 480) # set video height

            while True:
                if self.stop:
                    self.socket.disconnect()
                    break

                ret, img =cam.read()
                gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

                faces = faceCascade.detectMultiScale( 
                    gray,
                    scaleFactor = 1.2,
                    minNeighbors = 3,
                   )
                
#                 after detecting faces from the frame its time for predicting the faces
                for(x,y,w,h) in faces:
                    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
                    id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

                    # If confidence is less then 100 ==> "0" : perfect match 
                    if (confidence < 100):
                        sio.emit('push_faces', {'location': random.choice(locations),'name':names[id]}, namespace='/facemap')                    
                        id = names[id]
                        confidence = "  {0}%".format(round(100 - confidence))

                    else:
                        id = "unknown"
                        confidence = "  {0}%".format(round(100 - confidence))

                    cv2.putText(
                                img, 
                                str(id), 
                                (x+5,y-5), 
                                font, 
                                1, 
                                (255,255,255), 
                                2
                               )
                    cv2.putText(
                                img, 
                                str(confidence), 
                                (x+5,y+h-5), 
                                font, 
                                1, 
                                (255,255,0), 
                                1
                               )  

                cv2.imshow('camera',img) 
                if cv2.waitKey(10) & 0xff==ord("q"):
                    self.socket.disconnect()
                    break
                    # before ending its complusory we release resources 

        except Exception as e:
            print(e)
                
        cam.release()
        cv2.destroyAllWindows()
   

In [18]:
     

# Socket Manager

# sio = socketio.Client(logger=True)
sio = socketio.Client()


@sio.event
def connect_error():
    print("The connection failed!")    
    
@sio.event
def disconnect():
    print("disconnected")
    
# just for testing
@sio.on('new_faces', namespace='/facemap')
def new_faces(data):
    print("new faces")
    print(data)
    return "OK", 123

@sio.on('old_faces', namespace='/facemap')
def old_faces(data):
    print("old faces")
    print(data)
    return "OK", 123

@sio.on('param_error', namespace='/facemap')
def param_error(data):
    print("param_error")
    print(data)
    return "OK", 123

@sio.on('connect', namespace='/facemap')
def on_connect():
    print("Start Pushing the faces and Locations of people")

@sio.on('disconnect', namespace='/facemap')
def on_disconnect():
    print("on disconnect")
    _host
sio.connect(_host,headers={'Authorization':'TOKEN '+TOKEN}, namespaces=['/facemap'])    
# sio.connect('https://face-map-node-server.herokuapp.com',headers={'Authorization':'TOKEN '+TOKEN}, namespaces=['/facemap'])

In [19]:
# Recognizer
# sio.emit('push_faces', {'location': 'FoodCourt'}, namespace='/facemap')

Start Pushing the faces and Locations of people
old faces
[{'current_location': 'Tea Point', 'createdAt': '2021-01-07T04:37:15.009Z', '_id': '5ff68fe258b6400017bc1673', 'name': 'puneeth', '__v': 0}, {'current_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-07T09:10:21.795Z', '_id': '5ff6cffb27385700175cad0b', 'name': 'puneeth', '__v': 0}, {'current_location': 'Library', 'createdAt': '2021-01-07T09:10:19.323Z', '_id': '5ff6cffb27385700175cad0c', 'name': 'puneeth', '__v': 0}, {'current_location': 'CSE - BLOCK', 'createdAt': '2021-01-07T09:10:19.504Z', '_id': '5ff6cffb27385700175cad0d', 'name': 'puneeth', '__v': 0}, {'current_location': 'Aryabatta Block(IT)', 'createdAt': '2021-01-07T09:15:27.401Z', '_id': '5ff6d11b27385700175cad0e', 'name': 'puneeth', '__v': 0}, {'current_location': 'IT - BLOCK', 'createdAt': '2021-01-07T09:15:07.770Z', '_id': '5ff6d11b27385700175cad0f', 'name': 'puneeth', '__v': 0}, {'current_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-07

Recognizer has 
    * Train 
    * Recognize


In [20]:
recognizer = Recognizer(sio)
# recognizer.train()
recognizer.recognize()
# recognizer.capture()
# rtsp://192.168.0.5:8080/h264_pcm.sdp

initializing Recognizer
1
new faces
{'current_location': 'Swami Vivekananda Boys Hostel', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Swami Vivekananda Boys Hostel', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Swami Vivekananda Boys Hostel', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.67

new faces
{'current_location': 'Mahatma Gandhi Statue', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Mahatma Gandhi Statue', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Mahatma Gandhi Statue', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Aryabatta Block(IT)', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Aryabatta Block(IT)', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3

new faces
{'current_location': 'Tea Point', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'Library', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'Library', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'CSE - BLOCK', 'last_location': 'Library', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'C

new faces
{'current_location': 'Tea Point', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'Aryabatta Block(IT)', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'Aryabatta Block(IT)', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location'

new facesnew facesnew faces
{'current_location': 'Tea Point', 'last_location': 'Tea Point', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}


{'current_location': 'Tea Point', 'last_location': 'Tea Point', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
{'current_location': 'Tea Point', 'last_location': 'Tea Point', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852

...

new faces
{'current_location': 'Library', 'last_location': 'Aryabatta Block(IT)', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new facesnew faces
{'current_location': 'Library', 'last_location': 'Aryabatta Block(IT)', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}

{'current_location': 'Library', 'last_location': 'Aryabatta Block(IT)', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
new faces{'current_location': 'Aryabatta Block(IT)', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}new faces
{'current_location': 'Aryabatta Block(IT)', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 

new faces
{'current_location': 'Aryabatta Block(IT)', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Aryabatta Block(IT)', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Aryabatta Block(IT)', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Mahatma Gandhi Statue', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Mahatma Gandhi Statue', 'last_location': 'Swami Vivekananda Boys Hostel', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38

new faces
{'current_location': 'Dosa point', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Dosa point', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Dosa point', 'last_location': 'C V Raman Boys Hostel', 'createdAt': '2021-01-11T20:23:13.678Z', '_id': '5ffcb3b139ed3b00170e1d84', 'name': 'Priyanka Chopra', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'IT - BLOCK', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'IT - BLOCK', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Library', 'l

new faces
{'current_location': 'C V Raman Boys Hostel', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'C V Raman Boys Hostel', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Library', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new facesnew faces
{'current_location': 'Swami Vivekananda Boys Hostel', 'last_location': '

new faces
{'current_location': 'IT - BLOCK', 'last_location': 'Aryabatta Block(IT)', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Tea Point', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}new faces
{'current_location': 'Tea Point', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Library', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}

new faces
{'current_location': 'Library', 'last_location': 'Mahatma Gandhi Statue', 'createdAt': '2021-01-11T20:22:33.852Z', '_id': '5ffcb38939ed3b00170e1d80', 'name': 'Puneeth', '__v': 0}
new faces
{'current_location': 'Mahatma Gandhi Sta

In [10]:
# recognizer.disconnect()

In [129]:
#print(help(cv2.face))

In [22]:
recognizer.recognize()

1
on disconnect
disconnected
on disconnect
disconnected


Exception in thread Thread-642:
Traceback (most recent call last):
  File "D:\softwares\anaconda\lib\site-packages\socketio\client.py", line 280, in connect
Exception in thread Thread-641:
Traceback (most recent call last):
  File "D:\softwares\anaconda\lib\site-packages\socketio\client.py", line 280, in connect
        self.eio.connect(url, headers=headers, transports=transports,
  File "D:\softwares\anaconda\lib\site-packages\engineio\client.py", line 194, in connect
self.eio.connect(url, headers=headers, transports=transports,
  File "D:\softwares\anaconda\lib\site-packages\engineio\client.py", line 194, in connect
    return getattr(self, '_connect_' + self.transports[0])(
  File "D:\softwares\anaconda\lib\site-packages\engineio\client.py", line 300, in _connect_polling
    return getattr(self, '_connect_' + self.transports[0])(
  File "D:\softwares\anaconda\lib\site-packages\engineio\client.py", line 300, in _connect_polling
    raise exceptions.ConnectionError(
engineio.exception

NameError: name 'url' is not defined